In [15]:
!pip install numpy
!pip install pandas
!pip install seaborn


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: C:\Users\aanand\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: C:\Users\aanand\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: C:\Users\aanand\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [17]:
students = pd.read_csv("students.csv")
exams = pd.read_csv("exams.csv")
enrolments = pd.read_csv("enrolements.csv")
rooms = pd.read_csv("rooms.csv")
slots= 27

In [18]:
class Exam:
    def __init__(self, exam_code, name, duration, department_code):
        self.code = exam_code
        self.name = name
        self.duration = duration
        self.dep_code = department_code
        self.conflicts = 0
        self.priority = 0
        self.slot = -1
        self.students = []
        self.room = 0
        self.roomAssigned = False

class Student:
    def __init__(self, student_code, course_code):
        self.code = student_code
        self.course_code = course_code
        self.exams = []
        self.exam_codes = []

class Enrolment:
    def __init__(self, student_code, exam_code):
        self.student_code = student_code
        self.exam_code = exam_code

class Room:
    def __init__(self, room_code, capacity):
        self.code = room_code
        self.capacity = capacity
        self.status = []


In [19]:
examObjects = [Exam(**kwargs) for kwargs in exams.to_dict(orient='records')]

studentObjects = [Student(**kwargs) for kwargs in students.to_dict(orient='records')]

enrolmentObjects = [Enrolment(**kwargs) for kwargs in enrolments.to_dict(orient='records')]

roomObjects = [Room(**kwargs) for kwargs in rooms.to_dict(orient='records')]

In [20]:
for i in range(len(enrolmentObjects)):
    for j in range(len(studentObjects)):
        if studentObjects[j].code == enrolmentObjects[i].student_code:
            studentObjects[j].exam_codes.append(enrolmentObjects[i].exam_code)

In [21]:
for i in range(len(examObjects)):
    for j in range(len(studentObjects)):
        for k in studentObjects[j].exam_codes:
            if k == examObjects[i].code:
                studentObjects[j].exams.append(examObjects[i])
                examObjects[i].students.append(studentObjects[j])

In [22]:
for i in range(len(roomObjects)):
    for j in range(slots):
        roomObjects[i].status.append(False)

In [23]:
def degreeOfConflict(exam):
    flagExam = enrolmentObjects[0]
    degree = 0
    for i in enrolmentObjects:
        if i.exam_code == exam.code:
            flagExam = i
            
    for j in enrolmentObjects:
        if j.exam_code != flagExam.exam_code and j.student_code == flagExam.student_code:
            degree += 1
    return degree

In [24]:
def partialExamOrdering():
    sup = np.zeros(len(examObjects))
    for i in range(len(examObjects)):
        for j in range(len(examObjects)):
            if degreeOfConflict(examObjects[i]) > degreeOfConflict(examObjects[j]):
                sup[i] = sup[i] + 1
                continue
            elif degreeOfConflict(examObjects[i]) < degreeOfConflict(examObjects[j]):
                sup[j] = sup[j] + 1
                continue
            if len(examObjects[i].students) > len(examObjects[j].students):
                sup[i] = sup[i] + 1
                continue
            elif len(examObjects[i].students) < len(examObjects[j].students):
                sup[j] = sup[j] + 1
                continue
            if examObjects[i].duration > examObjects[j].duration:
                sup[i] = sup[i] + 1
                continue
            elif examObjects[i].duration < examObjects[j].duration:
                sup[j] = sup[j] + 1
                continue
            if examObjects[i].code > examObjects[j].code:
                sup[j] = sup[j] + 1
                continue
            elif examObjects[i].code < examObjects[j].code:
                sup[i] = sup[i] + 1
                continue
    return sup            
                    

In [25]:
'''def conflicting(exam1, exam2):
    conflicting = False
    if exam1.code != exam2.code:
        for i in exam1.students:
            for j in exam2.students:
                if i.code == j.code:
                    conflicting = True
    return conflicting
'''

'def conflicting(exam1, exam2):\n    conflicting = False\n    if exam1.code != exam2.code:\n        for i in exam1.students:\n            for j in exam2.students:\n                if i.code == j.code:\n                    conflicting = True\n    return conflicting\n'

In [26]:
def totalConflictsForExams():
    partialExamOrder = partialExamOrdering()
    for i in range(len(examObjects)):
        examObjects[i].conflicts = degreeOfConflict(examObjects[i])
        examObjects[i].priority = partialExamOrder[i]
        

In [27]:
def getKey(obj):
    return obj.priority

In [28]:
EVAPORATION_RATE = 0.3;
numAnts = 3;
t_max = 3.3;
t_min = 0.019;
MAX_ITER = 10000
MAX_TIME_SEC = 10
alpha = 1.0
beta = 1.0

In [29]:
pheromoneMatrix = []

def updatePheromoneMatrix():
    for i in range(len(examObjects)):
        for j in range(slots):
            if examObjects[i].slot == j:
                pheromoneMatrix[i][j] = (1 - EVAPORATION_RATE) * pheromoneMatrix[i][j] + 1
            else:
                pheromoneMatrix[i][j] = (1 - EVAPORATION_RATE) * pheromoneMatrix[i][j]
            if pheromoneMatrix[i][j] < t_min:
                pheromoneMatrix[i][j] = t_min
            elif pheromoneMatrix[i][j] > t_max:
                pheromoneMatrix[i][j] = t_max
    
def initPheromoneMatrix(exams, slots, value):
        for i in range(exams.size):
            pheromoneMatrix.append([value for j in range(slots)])
    

In [30]:
heuristicsMatrix = []

def initHeuristicsMatrix(exams, slots):
    for i in range(0, len(exams)):
        heuristicsMatrix.append([1 for j in range(slots)])

In [31]:
def assignRooms():
    for i in examObjects:
        for j in roomObjects:
                if not i.roomAssigned and len(i.students) < j.capacity and j.status[i.slot] == False:
                    i.room = j
                    i.roomAssigned = True
                    j.status[i.slot] = True
            

In [32]:
def checkForHardConstraintViolations(exam, slot):
    score = 0
    for i in range(len(studentObjects)):
        for j in studentObjects[i].exams:
            if exam.code == j.code and j.slot != -1 and j.slot == slot:
                score += 16
    return score      

In [33]:
def checkForSoftConstraintViolations(exam, slot):
    score = 0
    for i in range(len(studentObjects)):
        for j in studentObjects[i].exams:
            if exam.code == j.code and j.slot != -1 and j.slot > slot - 6:
                score = 4
    return score

In [34]:
def checkForSaftConstraintViolations(exam, slot):
    score = 0
    for i in range(len(studentObjects)):
        for j in studentObjects[i].exams:
            if exam.code == j.code and j.slot != -1 and j.slot > slot - 6:
                score = 4
    return score

In [35]:
def findForbiddenSlots(exam):
    slotters = set()
    for i in exam.students:
        for j in i.exams:
            if j.slot != -1:
                slotters.add(j.slot)
    return slotters

In [36]:
def numAdditionalViolations(exam, slot):
    score = 0
    score += checkForHardConstraintViolations(exam, slot)
    score += checkForSoftConstraintViolations(exam, slot)
    return score

In [37]:
def chooseSlot2(antIndex, examIndex, exam, slots):
    sum_desirable = 0.0
    desirability = np.zeros(slots)
    cList = set(range(27))
    cList = cList - findForbiddenSlots(exam)
    if exam.slot != -1:
        cList = cList - {exam.slot}
    for i in cList:
        heuristicsMatrix[examIndex][i] = 1 / (1 + numAdditionalViolations(exam, i))
        desirability[i] = ((pheromoneMatrix[examIndex][i]**alpha) * (heuristicsMatrix[examIndex][i]**beta))/pheroHeuristicSummation(examIndex, i)
        sum_desirable += desirability
    #print(exam.code + ": " + str(desirability))
    return np.argmax(desirability)

In [38]:
def pheroHeuristicSummation(examIndex, slotIndex):
    sum = 0
    for j in range(slots):
        if j != slotIndex:
            sum = (pheromoneMatrix[examIndex][j]**alpha) * (heuristicsMatrix[examIndex][j]**beta)
    return sum

In [39]:
index = 0
initPheromoneMatrix(exams, slots, t_max)
initHeuristicsMatrix(exams, slots)
totalConflictsForExams()
sortedExamsByPriority = sorted(examObjects, key=getKey, reverse=True)
while index < 10:
    for k in range(numAnts):
        array = []
        chosenSlot = 0
        for i in range(len(examObjects)):
            chosenSlot = chooseSlot2(k, i,examObjects[i], slots)
            examObjects[i].slot = chosenSlot
            #print(examObjects[i].code + ": " + str(examObjects[i].slot))
    assignRooms()
    updatePheromoneMatrix()
    index += 1

In [ ]:
for i in examObjects:
    print(i.code + " : " + str(i.slot) + " : " + str(i.room.code))

In [ ]:
for i in examObjects:
    for j in examObjects:
        